# Convert from GeoPackage to AiGIS format file (Multi-wavelength)
- Input: NIRS3 L2d vector data (GeoPackage)
- Output: Multi-value for one polygon AiGIS format

In [1]:
import numpy as np
import math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, GeometryCollection
from shapely.ops import split

In [2]:
def read_obj_file(file_path):
    # lists for vertices and faces
    vertices = []
    faces = []

    # Open shape model OBJ file
    with open(file_path, 'r') as file:
        for line in file:
            # Vertex information starts from 'v'
            if line.startswith('v '):
                # Store xyz coordinate of vertices
                vertex = list(map(float, line.split()[1:])) 
                vertices.append(vertex)

            # Face information starts from 'f'
            elif line.startswith('f '):
                # Store the indeces which compose the face
                face = [int(idx.split('/')[0]) - 1 for idx in line.split()[1:]]  # '/'で分割してインデックスのみ取得
                faces.append(face)

    return vertices, faces

def create_polygon(coords):
    longitudes = [lon for lon, lat in coords]
    difference  = max(longitudes) - min(longitudes)
    adjust_lon = (difference >= 300)
    adjusted_coords = [(lon - 360 if lon > 300 and adjust_lon else lon, lat) for lon, lat in coords]
    return Polygon(adjusted_coords)

# Split a polygon if the polygon intersect with longitude 0 degree
def split_polygon_at_line(polygon, line):
    result = []
    try:
        if polygon.intersects(line):
            split_result = split(polygon, line)
            if isinstance(split_result, GeometryCollection):
                for part in split_result.geoms:
                    if not part.is_empty:
                        result.append(part)
            else:
                if not split_result.is_empty:
                    result.append(split_result)

    except Exception as e:
        print(f"Error in split_polygon_at_line: {e}")

    return result

line = LineString([(0, -90), (0, 90)])

In [6]:
# Input file: NIRS3 L2d GeoPackage data
vector_path = './data/hyb2_nirs3_20180711_01_l2d_geo.gpkg'

#Output file: AiGIS format file
day = vector_path.split('_')[-4]
out_folder = './output/'
out_file =  f'nirs3_{day}_l2dmap.txt'
out_path1 = out_folder + out_file
out_path2 = out_folder + out_file.replace('.txt','_count.txt')

print("Input file: ", vector_path.split('/')[-1])
print("Output file: ", out_file)

# Shape model OBJ file
file_path = './data/SHAPE_SPC_49k_v20200323.obj' 

# Set NIRS3 spectral wavelength (micron) (ref. SIS document)
n=np.arange(128)+1
wl_all=(1230.33+18.5651*n-0.00492138*n**2)/1000.


Input file:  hyb2_nirs3_20180711_01_l2d_geo.gpkg
Output file:  nirs3_20180711_l2dmap.txt


In [7]:
# Read vertices and faces from the shape model OBJ file
vertices, faces = read_obj_file(file_path)

# Open input vector file (geopackage)
gdf = gpd.read_file(vector_path)

#convert xyz coordinates to map coordinates
root_xy = [math.sqrt(vertex[0]**2 + vertex[1]**2) for vertex in vertices]
root_xyz = [math.sqrt(vertex[0]**2 + vertex[1]**2 + vertex[2]**2) for vertex in vertices]

cos = [vertices[0] / r_xy if vertices[1] > 0 else vertices[0] / r_xy * (-1) for vertices, r_xy in zip(vertices, root_xy)]
sin = [vertices[2] / r_xyz for vertices, r_xyz in zip(vertices, root_xyz)]

lon = [(math.acos(c) * 180 / math.pi) if vertices[1] > 0 else (180 + math.acos(c) * 180 / math.pi)
    for vertices, c in zip(vertices, cos)]
lon_np = np.array(lon)

lat = [(math.asin(s) * 180 / math.pi) for s in sin]
lat_np = np.array(lat)

face_coordinates = []
average_result = []
count_result = []

f = 0
for face in faces:
    coordinates = []
    for idx in face:
        lon_val = lon[idx]
        lat_val = lat[idx]

        coordinates.append((lon_val,lat_val))
    face_coordinates.append(coordinates)

    if coordinates:
        coordinates.append(coordinates[0])

    #座標リストをpolygonにする
    poly = create_polygon(coordinates)
    try:
        split_polygons = split_polygon_at_line(poly, line)
    except Exception as e:
        print(f"Error splitting feature {f}: {e}")
        continue

    if not split_polygons:
        overlap = gdf[gdf.geometry.intersects(poly)]
    else:
        tmp1 = gdf[gdf.geometry.intersects(split_polygons[0])]
        tmp2 = gdf[gdf.geometry.intersects(split_polygons[1])]
        con = pd.concat([tmp1, tmp2])
        overlap = con.drop_duplicates().reset_index(drop=True)
        aa = 0

    columns_to_keep  = [col for col in overlap.columns if "wl" in col]

    if columns_to_keep:
        # 数値列のみを選択して平均値を計算
        ave_value = overlap[columns_to_keep].select_dtypes(include="number").mean(axis=0)
        count = len(overlap)

    else:
        print("No columns containing 'wl' were found.")
        ave_value = None
        count = None

    if isinstance(ave_value, pd.Series):
        ave_value_dict = ave_value.to_dict()
    else:
        ave_value_dict = eval(ave_value)

    values = list(ave_value_dict.values())

    average_result.append(values)
    count_result.append(count)

    # 各面の座標リストをface_coordinatesに追加
    face_coordinates.append(coordinates)

    f = f + 1

In [8]:
#Output AiGIS format file
with open(out_path1, 'w') as f:
    # Write AiGIS header
    f.write("NIRS_COUNT\n")
    f.write("reflectance[-]\n")
    f.write("wavelength[micron]\n")
    f.write(f"{len(average_result)}\n")
    f.write(f"_ ")
    wl = " ".join([f"{value:.6f}" for value in wl_all[30:103]])
    f.write(wl + "\n")

    # Write reflectance value from 1.8 to 3.1 um.
    min_wl = wl_all[30]
    max_wl = wl_all[103]

    ii=1
    for i in average_result:
        indices = [k for k, wl in enumerate(wl_all) if min_wl <= wl <= max_wl]
        selected_reflectance = [i[k] for k in indices]

        formatted_reflectance = " ".join(
            "0.0" if str(r).strip().lower() == 'nan' else f"{float(r):.6f}"
            for r in selected_reflectance
        )

        f.write(f"{ii} {formatted_reflectance}\n")
        ii = ii+1


ii=1
with open(out_path2, 'w') as f:
    # Write AiGIS header
    f.write("NIRS_COUNT\n")
    f.write("-\n")
    f.write(f"{len(average_result)}\n")
    for i in count_result:
        f.write(f"{ii} {i:.6f}\n")
        ii=ii+1

print('complete!')

complete!
